<a href="https://colab.research.google.com/github/trucareer/myrepo/blob/main/A_Data_Analyst_Agent_Built_with_Cohere_and_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install --quiet langchain langchain_cohere langchain_experimental

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.3/204.3 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.8/207.8 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.8/389.8 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.4/140.4 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 75.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.

# **Setup**

In [ ]:
### LLMs
import os
os.environ["COHERE_API_KEY"] = "cAwYIaRVHExcA53qCBYvJjp6hcTO33UhjJ12Ksp6"

In [ ]:
from langchain_cohere import ChatCohere
chat = ChatCohere(model="command-r-plus", temperature=0.3)

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


Our simple data analyst will be equipped with:
- Web search tool
- Python interpreter

# **Web search**

Use the Tavily API

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

os.environ["TAVILY_API_KEY"] = "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"

internet_search = TavilySearchResults()
internet_search.name = "Internet Search"
internet_search.description = "Return a list of relevant document snippets for a textual query retrieved from the internet."


from langchain_core.pydantic_v1 import BaseModel, Field
class TavilySearchInput(BaseModel):
    query: str = Field(description="Query to search the internet with")
internet_search.args_schema = TavilySearchInput

# **Python Interpreter Tool**

In [ ]:
from langchain.agents import Tool
from langchain_experimental.utilities import PythonREPL

python_repl = PythonREPL()
repl_tool = Tool(
    name="python_repl",
    description="Execute python code and retrun the result. The code runs in a static without interactive mode, so print output or save output to a file.",
    func=python_repl.run,
)
repl_tool.name = "python_interpreter"

class ToolInput(BaseModel):
  code: str = Field(description="Python code to execute.")
repl_tool.args_schema = ToolInput

In [ ]:
from langchain.agents import AgentExecutor
from langchain_cohere.react_multi_hop.agent import create_cohere_react_agent
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
prompt = ChatPromptTemplate.from_template("{input}")

agent = create_cohere_react_agent(
    llm = chat,
    tools = [internet_search, repl_tool],
    prompt = prompt,
)

In [ ]:
agent_executer = AgentExecutor(agent=agent, tools=[internet_search, repl_tool], verbose=True)

In [15]:
agent_executer.invoke({
    "input": "Create a plot of the number of full time employee at the 3 tech companies with the highest market cap in the India in 2024"
})



> Entering new AgentExecutor chain...

First, I will search for the 3 tech companies with the highest market cap in India in 2024. Then, I will search for the number of full-time employees at each of these companies. Finally, I will use Python to create a plot of the number of full-time employees at these companies.
{'tool_name': 'Internet Search', 'parameters': {'query': '3 tech companies with highest market cap in India 2024'}}
[{'url': 'https://ceoworld.biz/2024/02/17/indias-largest-companies-by-market-capitalization-2024/', 'content': "India's Largest Companies by market capitalization, 2024. Despina Wilson Banking and Finance February 17, 2024. As of August 13, 2024, Reliance Industries is the most valuable company in India, with a market cap of $235 billion. Following it are Tata Consultancy Services ($181 billion) and HDFC Bank Limited ($151 billion)."}, {'url': 'https://www.knowledgehut.com/blog/web-development/top-it-companies-in-india', 'content': "In recent news, TCS was r

{'input': 'Create a plot of the number of full time employee at the 3 tech companies with the highest market cap in the India in 2024',
 'output': "Here is a plot of the number of full-time employees at the 3 tech companies with the highest market cap in India in 2024:\n\n! [Number of Full-Time Employees at Top 3 Tech Companies in India by Market Cap in 2024]('number_of_full_time_employees_at_top_3_tech_companies_in_india_by_market_cap_in_2024.png')\n\nThe number of full-time employees at these companies ranges from around 140,000 to over 600,000. Tata Consultancy Services has the highest number of full-time employees, followed by HDFC Bank Limited and Reliance Industries.",
 'citations': [CohereCitation(start=123, end=662, text="! [Number of Full-Time Employees at Top 3 Tech Companies in India by Market Cap in 2024]('number_of_full_time_employees_at_top_3_tech_companies_in_india_by_market_cap_in_2024.png')\n\nThe number of full-time employees at these companies ranges from around</co:

In [16]:
agent_executer.invoke({
    "input": "Create a plot of the number of full time employee at the 3 tech companies with the highest market cap in the World in 2024"
})



> Entering new AgentExecutor chain...

First, I will search for the 3 tech companies with the highest market cap in 2024. Then, I will search for the number of full-time employees at each of these companies. Finally, I will use Python to create a plot of the data.
{'tool_name': 'Internet Search', 'parameters': {'query': '3 tech companies with highest market cap 2024'}}
[{'url': 'https://coveragely.com/blog/metrics-and-analysis/the-top-100-tech-companies-2024/', 'content': "Top 100 tech companies in the world in 2024. Top 10 Tech Companies In The World In 2024. 1. Alphabet (Google) ... Apple is an industry pioneer, with the world's second-highest market cap at $3.28 trillion. This dominance extends to revenue generation, where Apple pulls in a remarkable $382 billion, solidifying its position as the second-largest ..."}, {'url': 'https://investingnews.com/daily/tech-investing/top-technology-stocks/', 'content': 'Apple became the first company to surpass a market cap of US$3 trillion i

{'input': 'Create a plot of the number of full time employee at the 3 tech companies with the highest market cap in the World in 2024',
 'output': "Here is a plot of the number of full-time employees at the 3 tech companies with the highest market cap in 2024:\n\n! [Number of Full-Time Employees at Top 3 Tech Companies in 2024]('number_of_full_time_employees_at_top_3_tech_companies_in_2024.png')\n\nThe data shows that Microsoft has the highest number of full-time employees, followed by Alphabet (Google), and then Apple.",
 'citations': [CohereCitation(start=114, end=248, text="! [Number of Full-Time Employees at Top 3 Tech Companies in 2024]('number_of_full_time_employees_at_top_3_tech_companies_in_2024.png')", documents=[{'url': 'https://coveragely.com/blog/metrics-and-analysis/the-top-100-tech-companies-2024/', 'content': "Top 100 tech companies in the world in 2024. Top 10 Tech Companies In The World In 2024. 1. Alphabet (Google) ... Apple is an industry pioneer, with the world's se

In [17]:
agent_executer.invoke({
    "input": "How r u?"
})



> Entering new AgentExecutor chain...

I will answer the user's question directly.
{'tool_name': 'directly_answer', 'parameters': {}}
directly_answer is not a valid tool, try one of [Internet Search, python_interpreter].Relevant Documents: None
Cited Documents: None
Answer: I'm an AI assistant, so I don't have feelings as such, but I'm here to help you with your queries. How can I help?
Grounded answer: I'm an AI assistant, so I don't have feelings as such, but I'm here to help you with your queries. How can I help?

> Finished chain.


{'input': 'How r u?',
 'output': "I'm an AI assistant, so I don't have feelings as such, but I'm here to help you with your queries. How can I help?",
 'citations': []}